<a href="https://colab.research.google.com/github/10firestorm/repo1/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os
!pip install -U gradio
import gradio as gr

!pip install -U langchain-community langchain-core fastapi uvicorn requests beautifulsoup4 pymupdf faiss-cpu huggingface_hub transformers accelerate bitsandbytes langchain-openai gradio

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('read')

from langchain.document_loaders import WebBaseLoader, PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.chains.question_answering import load_qa_chain

def load_pdf(file_path):
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()
    return docs

def load_website(url):
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs

def setup_vector_store(docs):
    embeddings = HuggingFaceEmbeddings()
    vector_store = FAISS.from_documents(docs, embeddings)
    return vector_store

def ask_question(query, vector_store):
    retriever = vector_store.as_retriever()
    docs = retriever.get_relevant_documents(query)
    llm = HuggingFaceHub(
        repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
        model_kwargs={"temperature": 0.7, "max_length": 512}
    )
    chain = load_qa_chain(llm, chain_type="stuff")
    response = chain.run(input_documents=docs, question=query)
    return response

def process_input(weblink, pdf_file, question):
    docs = []
    if weblink:
        docs.extend(load_website(weblink))
    if pdf_file:
        docs.extend(load_pdf(pdf_file.name))

    if not docs:
        return "Please provide a website link or upload a PDF."

    vector_store = setup_vector_store(docs)
    response = ask_question(question, vector_store)
    return response

demo = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Textbox(label="Website Link (Optional)"),
        gr.File(label="Upload PDF (Optional)"),
        gr.Textbox(label="Ask a Question")
    ],
    outputs=gr.Textbox(label="Final Answer"),
    title="Web & PDF QA System"
)

demo.launch()
